In [ ]:
!pip install speechbrain torchaudio==0.9.0 transformers

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
from pathlib import Path

import torch
import torchaudio
from speechbrain.pretrained import SpectralMaskEnhancement
from tqdm import tqdm

In [3]:
enhance_model = SpectralMaskEnhancement.from_hparams(
    source='speechbrain/metricgan-plus-voicebank',
    savedir='pretrained_models/metricgan-plus-voicebank',
    run_opts={'device': 'cuda'},
)

Downloading:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/7.59M [00:00<?, ?B/s]

In [7]:
!cp drive/MyDrive/val.zip . && unzip -qq val.zip && rm val.zip && ls -l val | wc -l

1001


In [10]:
res_dir = Path('res')
res_dir.mkdir(
    parents=True,
    exist_ok=True,
)

In [13]:
for file in tqdm(Path('val').iterdir()):
    if file.suffix == '.wav':
        noisy = enhance_model.load_audio(str(file.resolve())).unsqueeze(0)
        enhanced = enhance_model.enhance_batch(
            noisy,
            lengths=torch.tensor([1.]),
        )
        torchaudio.save(
            res_dir / file.name,
            enhanced.cpu(),
            16000,
        )

1000it [01:01, 16.19it/s]


In [15]:
!zip -q -r speechbrain.zip res/ && mv speechbrain.zip drive/MyDrive/ && rm -rf res/